# TM forward prediction pass

Будем разбирать разные простые последовательности символов. Прямой путь предсказаний - последовательный переход на N шагов вперед по мета-последовательности, которая является суперпозицией различных последовательностей с общим началом. Например, если обучение производилось на последовательности "ABCD", то предсказание  на 2 шага вперед из "B" - это "D". А если обученеи производилось на последовательностях "ABCD" и "AKLM", то предсказание на 2 шага из "A" должно дать некоторую суперпозицию символов "C" и "L" (=объединение через OR).
В случае более сложных наборов тренировочных последовательностей и в зависимости от длины и количества разных контекстов, которые TM способна учитывать (зависит напрямую от параметра CellsPerColumn), суперпозиции могут быть сложнее, а так же попросту иметься/отсутствовать, интерферировать контекстами. Пока что с этим особо разбираться не буду - чуть-чуть пощупаю.

Цель:

- научиться работать с ТМкой и разобраться в некоторых ее базовых параметрах
- научиться делать раскрутку предсказания на несколько шагов вперед
- визуализировать и дебажить ТМку
- сделать простенький пайплайн для простых тестов на будущее (для бэктрекинга)

In [187]:
import numpy as np
from htm.bindings.sdr import SDR
from htm.algorithms import TemporalMemory as TM

def format_sdr(sdr, layer_ind=0):
    result = ''
    
    layer = sdr.dense
    if len(layer.shape) == 2:
        layer = sdr.dense[:, layer_ind]

    size = min(input_size, col_size * 5)
    for i in range(size):
        if i > 0 and i % col_size == 0:
            result += ' '
        result += str(layer[i])
    return result

def print_formatted_active_cells(val, active_cells):
    print(f'{val:<4} | {format_sdr(active_cells)} Active')
    for layer_ind in range(1, cells_per_column):
        print(f'     | {format_sdr(active_cells, layer_ind)}')
    
def print_formatted_predictive_cells(anomaly, predictive_cells):
    print(f'{anomaly:.2f} | {format_sdr(predictive_cells)} Predicted')
    for layer_ind in range(1, cells_per_column):
        print(f'     | {format_sdr(predictive_cells, layer_ind)}')

def encode_sdr_val(val):
    data = np.zeros(input_size, dtype=np.int8)
    fr = val * col_size
    to = fr + col_size
    data[fr:to] = 1
    return data

def train_cycle(arr, tm, input_sdr, print_enabled=False, learn=True, reset_enabled=True):
    if reset_enabled:
        tm.reset()
    
    for val in arr:
        input_sdr.dense = encode_sdr_val(val)
        tm.compute(input_sdr, learn=learn)
        if print_enabled:
            print_formatted_active_cells(val, tm.getActiveCells())
    
        tm.activateDendrites(learn)
        if print_enabled:
            print_formatted_predictive_cells(tm.anomaly, tm.getPredictiveCells())
            
def get_columns_from_cells(cells_sdr, tm):
    return list(sorted(set(tm.columnForCell(i) for i in cells_sdr.sparse)))

def get_vals_from_cells(cols):
    return list(sorted(set(col // col_size for col in cols)))

def predict_cycle(start_val, n_steps, tm, input_sdr, print_enabled=False, learn=False, reset_enabled=True):
    if reset_enabled:
        tm.reset()
    val = start_val
    input_sdr.dense = encode_sdr_val(val)
    for i in range(n_steps):
        tm.compute(input_sdr, learn=learn)
        if print_enabled:
            print_formatted_active_cells(' ', tm.getActiveCells())
    
        tm.activateDendrites(learn)
        prediction = tm.getPredictiveCells()
        if print_enabled:
            print_formatted_predictive_cells(tm.anomaly, prediction)
                
        input_sdr.sparse = get_columns_from_cells(prediction, tm)
        print(get_vals_from_cells(get_columns_from_cells(prediction, tm)))

## Cells per column = 1

In [205]:
input_size = 200
col_size = 20
cells_per_column = 1
input_sdr = SDR(input_size)

tm = TM(
    columnDimensions = (input_sdr.size,),
    # default: 32
    cellsPerColumn=cells_per_column,
    # default: 10
    minThreshold=12,
    # default: 13
    activationThreshold=18,
    # default: 0.21
    initialPermanence=0.5,
)

seqs = [
    # working
#     [0, 1, 3, 2],
    
    # working
#     [0, 1, 3],
#     [0, 2, 4],
    
    # not working
#     [0, 1, 3, 3, 2],
    
    # not working
    [0, 1, 3, 3, 4],
    [0, 2, 3, 3, 2],
]

for cycle in range(len(seqs) * 40):
    seq = seqs[np.random.choice(len(seqs))]
    train_cycle(seq, tm, input_sdr, print_enabled=False, learn=True, reset_enabled=True)

predict_cycle(0, 7, tm, input_sdr, print_enabled=True, learn=False, reset_enabled=True)
print('---------------------------------')

train_cycle([], tm, input_sdr, print_enabled=True, learn=False)
predict_cycle(1, 2, tm, input_sdr, print_enabled=True, learn=False, reset_enabled=False)

     | 11111111111111111111 00000000000000000000 00000000000000000000 00000000000000000000 00000000000000000000 Active
1.00 | 00000000000000000000 11111111111111111111 11111111111111111111 00000000000000000000 00000000000000000000 Predicted
[1, 2]
     | 00000000000000000000 11111111111111111111 11111111111111111111 00000000000000000000 00000000000000000000 Active
0.00 | 00000000000000000000 00000000000000000000 00000000000000000000 11111111111111111111 00000000000000000000 Predicted
[3]
     | 00000000000000000000 00000000000000000000 00000000000000000000 11111111111111111111 00000000000000000000 Active
0.00 | 00000000000000000000 00000000000000000000 11111111111111111111 11111111111111111111 11111111111111111111 Predicted
[2, 3, 4]
     | 00000000000000000000 00000000000000000000 11111111111111111111 11111111111111111111 11111111111111111111 Active
0.00 | 00000000000000000000 00000000000000000000 11111111111111111111 11111111111111111111 11111111111111111111 Predicted
[2, 3, 4]
     

## Cells per column = 2

In [214]:
input_size = 200
col_size = 20
cells_per_column = 2
input_sdr = SDR(input_size)

tm = TM(
    columnDimensions = (input_sdr.size,),
    # default: 32
    cellsPerColumn=cells_per_column,
    # default: 10
    minThreshold=12,
    # default: 13
    activationThreshold=18,
    # default: 0.21
    initialPermanence=0.5,
)

seqs = [
    # working
#     [0, 1, 3, 3, 2],
    
    # working
#     [0, 1, 3, 3, 4],
#     [0, 2, 3, 3, 2],
    
    # not working
#     [0, 1, 3, 3, 3, 2],
    
    # not working    
    [0, 1, 3, 3, 1, 3, 3, 4],
    [0, 2, 3, 3, 2, 3, 3, 2],
]

for cycle in range(len(seqs) * 80):
    seq = seqs[np.random.choice(len(seqs))]
    train_cycle(seq, tm, input_sdr, print_enabled=False, learn=True, reset_enabled=True)

predict_cycle(0, 6, tm, input_sdr, print_enabled=True, learn=False, reset_enabled=True)

print('------------------------------------')
train_cycle([], tm, input_sdr, print_enabled=True, learn=False)
predict_cycle(1, 7, tm, input_sdr, print_enabled=True, learn=False, reset_enabled=False)

     | 11111111111111111111 00000000000000000000 00000000000000000000 00000000000000000000 00000000000000000000 Active
1.00 | 00000000000000000000 11100101011011000111 01111001111010101000 00000000000000000000 00000000000000000000 Predicted
     | 00000000000000000000 00011010100100111000 10000110000101010111 00000000000000000000 00000000000000000000
[1, 2]
     | 00000000000000000000 11100101011011000111 01111001111010101000 00000000000000000000 00000000000000000000 Active
0.00 | 00000000000000000000 00000000000000000000 00000000000000000000 11101111011111001011 00000000000000000000 Predicted
     | 00000000000000000000 00000000000000000000 00000000000000000000 11110101111111110111 00000000000000000000
[3]
     | 00000000000000000000 00000000000000000000 00000000000000000000 11101111011111001011 00000000000000000000 Active
0.00 | 00000000000000000000 10001101010010000000 00000000000000000000 10111111111111111100 01111100100001110000 Predicted
     | 00000000000000000000 01110010101101

## Cells per column > 3

In [218]:
input_size = 200
col_size = 20
cells_per_column = 8
input_sdr = SDR(input_size)

tm = TM(
    columnDimensions = (input_sdr.size,),
    # default: 32
    cellsPerColumn=cells_per_column,
    # default: 10
    minThreshold=12,
    # default: 13
    activationThreshold=18,
    # default: 0.21
    initialPermanence=0.5,
)

seqs = [
    # working
#     [0, 1, 3, 3, 2],
    
    # working
#     [0, 1, 3, 3, 4],
#     [0, 2, 3, 3, 2],
    
    # not working
#     [0, 1, 3, 3, 3, 2],
    
    # not working    
    [0, 1, 3, 3, 1, 3, 4, 4],
    [0, 2, 3, 3, 2, 3, 3, 2],
]

for cycle in range(len(seqs) * 80):
    seq = seqs[np.random.choice(len(seqs))]
    train_cycle(seq, tm, input_sdr, print_enabled=False, learn=True, reset_enabled=True)

predict_cycle(0, 10, tm, input_sdr, print_enabled=True, learn=False, reset_enabled=True)

print('------------------------------------')
train_cycle([], tm, input_sdr, print_enabled=True, learn=False)
predict_cycle(1, 10, tm, input_sdr, print_enabled=True, learn=False, reset_enabled=False)

     | 11111111111111111111 00000000000000000000 00000000000000000000 00000000000000000000 00000000000000000000 Active
1.00 | 00000000000000000000 00000000000000000000 00000000000010100000 00000000000000000000 00000000000000000000 Predicted
     | 00000000000000000000 00000000001000001000 00000000000100000000 00000000000000000000 00000000000000000000
     | 00000000000000000000 01000000000000100000 01100000001000001000 00000000000000000000 00000000000000000000
     | 00000000000000000000 10100000000001000000 00000000000001000100 00000000000000000000 00000000000000000000
     | 00000000000000000000 00010000000000000100 00000000100000000000 00000000000000000000 00000000000000000000
     | 00000000000000000000 00001001000110000000 10000000000000000000 00000000000000000000 00000000000000000000
     | 00000000000000000000 00000000110000010011 00011001010000000000 00000000000000000000 00000000000000000000
     | 00000000000000000000 00000110000000000000 00000110000000010011 00000000000000000

In [212]:
print(format_sdr(tm.getWinnerCells()))
print(format_sdr(tm.getPredictiveCells()))

00000000000000000000 00000000000000000000 00000000000000000000 00000000000000000000 00000000000000000000
00000000000000000000 00000000000000000000 00000000000000000000 00000000000000000000 00000000000000000000


In [30]:
print(tm)

Temporal Memory Connections:
    Inputs (24) ~> Outputs (80) via Segments (24)
    Segments on Cell Min/Mean/Max 0 / 0.3 / 1
    Potential Synapses on Segment Min/Mean/Max 8 / 8 / 8
    Connected Synapses on Segment Min/Mean/Max 8 / 8 / 8
    Synapses Dead (0%) Saturated (0%)
    Synapses pruned (0%) Segments pruned (0%)
    Buffer for destroyed synapses: 0 	 buffer for destr. segments: 0

